# SARPYX SNAP Integration

This notebook demonstrates how to use SARPYX with ESA SNAP (Sentinel Application Platform) for advanced SAR data processing.

## Overview

SARPYX provides seamless integration with SNAP through:
- Direct SNAP operator access
- Graph Processing Tool (GPT) integration
- Custom processing workflows
- Automated parameter optimization

## Setup and Prerequisites

Before running this notebook, ensure:
1. ESA SNAP is installed on your system
2. SNAP's `gpt` tool is accessible from the command line
3. SARPYX SNAP module is properly configured

In [ ]:
import os
import sys
from pathlib import Path

# Import SARPYX SNAP integration
from sarpyx.snap import SNAPProcessor, GPTRunner
from sarpyx.utils import setup_logging, check_snap_installation

# Set up logging
setup_logging(level='INFO')

# Check SNAP installation
snap_available = check_snap_installation()
if snap_available:
    print("✓ SNAP installation detected")
else:
    print("⚠ SNAP not found. Please install SNAP and ensure 'gpt' is in your PATH")
    print("Download from: https://step.esa.int/main/download/snap-download/")

## Initialize SNAP Processor

The `SNAPProcessor` class provides a high-level interface to SNAP operations:

In [ ]:
# Initialize SNAP processor
snap_processor = SNAPProcessor()

# Configure processing parameters
snap_config = {
    'memory': '8G',  # Allocate 8GB memory to SNAP
    'parallel_processing': True,
    'temp_dir': '/tmp/sarpyx_snap',
    'cache_size': '2G'
}

snap_processor.configure(snap_config)
print("✓ SNAP processor configured")
print(f"Memory allocation: {snap_config['memory']}")
print(f"Temporary directory: {snap_config['temp_dir']}")

## Basic SNAP Operations

Let's start with basic SNAP operations on Sentinel-1 data:

In [ ]:
# Define input data path
input_path = Path("../../data/S1A_IW_SLC__1SDV_20231201T060000_20231201T060030_051234_062B5E_1234.SAFE")
output_dir = Path("outputs/snap_processing")
output_dir.mkdir(parents=True, exist_ok=True)

# Check if input data exists
if input_path.exists():
    print(f"✓ Input data found: {input_path.name}")
else:
    print("⚠ Input data not found. Using simulated processing.")
    input_path = None

### Read and Display Product Information

In [ ]:
if input_path and input_path.exists():
    # Read product information using SNAP
    product_info = snap_processor.get_product_info(input_path)
    
    print("Product Information:")
    print(f"  Name: {product_info.get('name', 'N/A')}")
    print(f"  Product Type: {product_info.get('product_type', 'N/A')}")
    print(f"  Mission: {product_info.get('mission', 'N/A')}")
    print(f"  Acquisition Date: {product_info.get('acquisition_date', 'N/A')}")
    print(f"  Polarizations: {product_info.get('polarizations', 'N/A')}")
    print(f"  Subswaths: {product_info.get('subswaths', 'N/A')}")
    print(f"  Scene Size: {product_info.get('scene_width', 'N/A')} x {product_info.get('scene_height', 'N/A')} pixels")
else:
    print("Simulating product information display:")
    print("Product Information:")
    print("  Name: S1A_IW_SLC__1SDV_20231201T060000_20231201T060030_051234_062B5E")
    print("  Product Type: SLC")
    print("  Mission: Sentinel-1A")
    print("  Acquisition Date: 2023-12-01T06:00:00")
    print("  Polarizations: ['VV', 'VH']")
    print("  Subswaths: ['IW1', 'IW2', 'IW3']")
    print("  Scene Size: 25000 x 16000 pixels")

## Processing Workflow with SNAP

Now let's create a complete processing workflow using SNAP operations:

In [ ]:
# Define processing workflow
workflow_steps = [
    'apply_orbit_file',
    'thermal_noise_removal', 
    'radiometric_calibration',
    'speckle_filtering',
    'terrain_correction'
]

# Set processing parameters for each step
processing_params = {
    'apply_orbit_file': {
        'orbit_type': 'Sentinel Precise (Auto Download)'
    },
    'thermal_noise_removal': {
        'removeThermalNoise': True
    },
    'radiometric_calibration': {
        'outputSigmaBand': True,
        'outputGammaBand': False,
        'outputBetaBand': False
    },
    'speckle_filtering': {
        'filter': 'Lee',
        'filterSize': '5x5'
    },
    'terrain_correction': {
        'demName': 'SRTM 1Sec HGT',
        'pixelSpacingInMeter': 10.0,
        'mapProjection': 'WGS84(DD)'
    }
}

print("Processing workflow defined:")
for i, step in enumerate(workflow_steps, 1):
    print(f"  {i}. {step.replace('_', ' ').title()}")

### Execute Processing Workflow

In [ ]:
if input_path and input_path.exists():
    print("Starting SNAP processing workflow...")
    
    # Execute the workflow
    try:
        output_path = snap_processor.run_workflow(
            input_path=input_path,
            output_path=output_dir / "processed_s1.dim",
            workflow_steps=workflow_steps,
            parameters=processing_params
        )
        
        print(f"✓ Processing completed successfully!")
        print(f"Output saved to: {output_path}")
        
    except Exception as e:
        print(f"✗ Processing failed: {str(e)}")
        
else:
    print("Simulating SNAP processing workflow execution...")
    print("[INFO] Reading product: S1A_IW_SLC__1SDV_20231201T060000_20231201T060030_051234_062B5E")
    print("[INFO] Applying orbit file...")
    print("[INFO] Removing thermal noise...")
    print("[INFO] Applying radiometric calibration...")
    print("[INFO] Applying speckle filtering...")
    print("[INFO] Performing terrain correction...")
    print("[INFO] Writing output product...")
    print("✓ Workflow simulation completed!")

## Advanced SNAP Integration

### Custom Graph Processing

In [ ]:
# Create a custom SNAP graph
from sarpyx.snap import GraphBuilder

# Initialize graph builder
graph_builder = GraphBuilder()

# Add nodes to the graph
graph_builder.add_node('Read', 'Read', {'file': '${input}'})
graph_builder.add_node('Apply-Orbit-File', 'Apply-Orbit-File', {
    'orbitType': 'Sentinel Precise (Auto Download)',
    'polyDegree': 3
})
graph_builder.add_node('Calibration', 'Calibration', {
    'outputSigmaBand': True,
    'selectedPolarisations': 'VV,VH'
})
graph_builder.add_node('Speckle-Filter', 'Speckle-Filter', {
    'filter': 'Lee',
    'filterSizeX': 5,
    'filterSizeY': 5
})
graph_builder.add_node('Terrain-Correction', 'Terrain-Correction', {
    'demName': 'SRTM 1Sec HGT',
    'pixelSpacingInMeter': 10.0
})
graph_builder.add_node('Write', 'Write', {
    'file': '${output}',
    'formatName': 'BEAM-DIMAP'
})

# Build the graph
graph_xml = graph_builder.build()

print("Custom SNAP graph created:")
print(f"Graph contains {len(graph_builder.nodes)} processing nodes")
print("\nGraph structure:")
for node in graph_builder.nodes:
    print(f"  - {node['id']}: {node['operator']}")

### Batch Processing with SNAP

In [ ]:
# Simulate batch processing multiple files
import glob

# Search for multiple input files (simulated)
input_pattern = "../../data/S1*.SAFE"
input_files = glob.glob(input_pattern)

if not input_files:
    # Simulate multiple files for demonstration
    input_files = [
        "S1A_IW_SLC__1SDV_20231201T060000_20231201T060030_051234_062B5E_1234.SAFE",
        "S1A_IW_SLC__1SDV_20231203T060000_20231203T060030_051236_062B60_5678.SAFE",
        "S1A_IW_SLC__1SDV_20231205T060000_20231205T060030_051238_062B62_9012.SAFE"
    ]
    print("Simulating batch processing with sample filenames")

print(f"Found {len(input_files)} files for batch processing:")
for i, file in enumerate(input_files, 1):
    filename = Path(file).name if isinstance(file, str) else file.name
    print(f"  {i}. {filename}")

# Configure batch processing
batch_config = {
    'max_parallel': 2,  # Process 2 files simultaneously
    'continue_on_error': True,
    'log_individual_results': True
}

print(f"\nBatch processing configured:")
print(f"  Max parallel processes: {batch_config['max_parallel']}")
print(f"  Continue on error: {batch_config['continue_on_error']}")

## Performance Monitoring

Monitor SNAP processing performance and resource usage:

In [ ]:
import time
import psutil
import matplotlib.pyplot as plt

# Simulate performance monitoring
def simulate_processing_metrics():
    """Simulate processing performance metrics"""
    times = list(range(0, 300, 10))  # 5 minutes of processing
    cpu_usage = [20 + 30 * (1 + 0.5 * (t/100)**2) * (1 + 0.2 * ((-1)**int(t/20))) for t in times]
    memory_usage = [2.1 + 1.5 * (t/300) + 0.3 * ((-1)**int(t/30)) for t in times]
    
    return times, cpu_usage, memory_usage

# Get simulated metrics
times, cpu_usage, memory_usage = simulate_processing_metrics()

# Plot performance metrics
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# CPU Usage
ax1.plot(times, cpu_usage, 'b-', linewidth=2, label='CPU Usage')
ax1.set_ylabel('CPU Usage (%)')
ax1.set_title('SNAP Processing Performance Monitoring')
ax1.grid(True, alpha=0.3)
ax1.legend()
ax1.set_ylim(0, 100)

# Memory Usage
ax2.plot(times, memory_usage, 'r-', linewidth=2, label='Memory Usage')
ax2.set_xlabel('Time (seconds)')
ax2.set_ylabel('Memory Usage (GB)')
ax2.grid(True, alpha=0.3)
ax2.legend()
ax2.set_ylim(0, 8)

plt.tight_layout()
plt.show()

print("✓ Performance monitoring simulation completed")
print(f"Average CPU usage: {sum(cpu_usage)/len(cpu_usage):.1f}%")
print(f"Peak memory usage: {max(memory_usage):.1f} GB")

## Error Handling and Troubleshooting

SARPYX provides comprehensive error handling for SNAP operations:

In [ ]:
from sarpyx.snap.exceptions import SNAPProcessingError, SNAPConfigurationError

def demonstrate_error_handling():
    """Demonstrate error handling capabilities"""
    
    try:
        # Simulate a processing error
        print("Simulating SNAP processing error...")
        raise SNAPProcessingError("Insufficient memory for processing large scene")
        
    except SNAPProcessingError as e:
        print(f"✗ Processing Error: {e}")
        print("Recommended solutions:")
        print("  1. Increase memory allocation in SNAP configuration")
        print("  2. Process smaller subsections of the scene")
        print("  3. Use more aggressive data tiling")
        
    try:
        # Simulate a configuration error
        print("\nSimulating SNAP configuration error...")
        raise SNAPConfigurationError("SNAP GPT executable not found in PATH")
        
    except SNAPConfigurationError as e:
        print(f"✗ Configuration Error: {e}")
        print("Recommended solutions:")
        print("  1. Verify SNAP installation")
        print("  2. Add SNAP bin directory to system PATH")
        print("  3. Set SNAP_HOME environment variable")

demonstrate_error_handling()
print("\n✓ Error handling demonstration completed")

## Summary

This notebook demonstrated:

1. **SNAP Integration Setup**: Configuring SARPYX to work with SNAP
2. **Basic Operations**: Reading product information and basic processing
3. **Workflow Processing**: Creating and executing complete processing workflows
4. **Advanced Features**: Custom graphs and batch processing
5. **Performance Monitoring**: Tracking processing performance and resource usage
6. **Error Handling**: Managing processing errors and troubleshooting

## Next Steps

- Explore **InSAR Processing** in `03_insar_processing.ipynb`
- Learn **Time Series Analysis** in `04_time_series_analysis.ipynb`
- Check out **Advanced Workflows** in `05_advanced_workflows.ipynb`

## Additional Resources

- [SNAP Documentation](https://step.esa.int/main/doc/)
- [SARPYX SNAP Module API](../api/snap.md)
- [Processing Examples](../examples/)